(5p) Read the files and build two large consolidate files that are the union of all the documents in 20N and BAC.

**SECCION PARA COLOCAR DONDE TIENES UBICADOS LOS DATASETS DE 20NEWS.ZIP Y BAC.ZIP**

In [ ]:
# CAMBIA estos nombres si el archivo está en otra ruta de tu PC/Colab
newsgroups_dataset = "20news-18828.tar.gz"  # Dataset 20 Newsgroups (formato .tar.gz). Fuente: http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz
bac_dataset = "BAC.zip"                      # Blog Authorship Corpus (formato .zip). Fuente: https://huggingface.co/datasets/barilan/blog_authorship_corpus

!mkdir -p datasets # Crea la carpeta 'datasets' si no existe (-p evita error si ya existe)

# Extrae el .tar.gz de 20 Newsgroups dentro de 'datasets'
!tar -xzf {newsgroups_dataset} -C datasets #  -x: extraer, -z: gzip, -f: archivo, -C DIR: destino de extracción

# Descomprime BAC.zip dentro de 'datasets'
!unzip -o {bac_dataset} -d datasets #  -o: sobrescribe archivos existentes, -d DIR: destino de extracción

# Descomprime blogs.zip generado por el paso anterior (queda en 'datasets')
!unzip -o "datasets/blogs.zip" -d datasets #  -o: sobrescribe si existe, -d DIR: destino de extracción

!mkdir -p large-files # Crea la carpeta para archivos grandes/concatenados (si no existe)

Streaming output truncated to the last 5000 lines.
  inflating: datasets/blogs/4002673.female.33.indUnk.Capricorn.xml  
  inflating: datasets/blogs/4002831.male.23.indUnk.Aquarius.xml  
  inflating: datasets/blogs/4002909.female.27.indUnk.Virgo.xml  
  inflating: datasets/blogs/4003064.male.15.Student.Capricorn.xml  
  inflating: datasets/blogs/4003080.female.16.indUnk.Cancer.xml  
  inflating: datasets/blogs/4003093.male.26.Student.Leo.xml  
  inflating: datasets/blogs/4003147.female.24.Tourism.Aries.xml  
  inflating: datasets/blogs/4003245.female.16.indUnk.Sagittarius.xml  
  inflating: datasets/blogs/4003260.male.24.Student.Pisces.xml  
  inflating: datasets/blogs/4003282.female.17.Student.Aquarius.xml  
  inflating: datasets/blogs/4003311.male.15.Student.Cancer.xml  
  inflating: datasets/blogs/4003433.female.17.Arts.Sagittarius.xml  
  inflating: datasets/blogs/4003773.female.15.Student.Gemini.xml  
  inflating: datasets/blogs/4003806.female.25.Engineering.Pisces.xml  
  inflatin

In [ ]:
import os
import sys
from pathlib import Path
import shutil
from typing import Generator


def iter_files(folder_path: str) -> Generator[Path, None, None]:
    """
    Recorre recursivamente un directorio y genera rutas de archivos encontrados.

    Args:
        folder_path (str): Ruta del directorio raíz desde el cual se inicia la búsqueda.

    Yields:
        Path: Objeto Path correspondiente a cada archivo encontrado.
    """
    stack: list[Path] = [Path(folder_path)]
    while stack:
        p: Path = stack.pop()
        for entry in os.scandir(p):
            if entry.is_dir(follow_symlinks=False):
                stack.append(Path(entry.path))
            elif entry.is_file(follow_symlinks=False):
                yield Path(entry.path)


def join_files_in_one_stream(
    folder_path: str,
    output_file_path: str,
    sep: bytes = b"\n"
) -> None:
    """
    Une todos los archivos de texto en un solo archivo binario.

    Los archivos se concatenan usando buffers grandes para minimizar
    la sobrecarga de Python. Cada archivo se separa con un delimitador.

    Args:
        folder_path (str): Directorio raíz que contiene los archivos a concatenar.
        output_file_path (str): Ruta del archivo de salida que contendrá el stream unido.
        sep (bytes, opcional): Separador a insertar entre archivos. Por defecto b"\\n".

    Raises:
        Exception: Si ocurre algún error al copiar un archivo.
    """
    out_path: Path = Path(output_file_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    with open(out_path, "wb") as out_f:
        for fpath in iter_files(folder_path):
            try:
                with open(fpath, "rb") as in_f:
                    shutil.copyfileobj(in_f, out_f, length=1024 * 1024)
                out_f.write(sep)
            except Exception as e:
                print(f"[WARN] No se pudo copiar {fpath}: {e}", file=sys.stderr)


if __name__ == "__main__":
    join_files_in_one_stream("datasets/20news-18828", "large-files/20news.txt")
    join_files_in_one_stream("datasets/blogs", "large-files/blogs.txt")

(5p) Tokenize by sentence.
- Normalize, but DO NOT eliminate stop words.
- Replace numbers with a token named NUM.
- Add sentence start and end tags "\<s>\</s>".
- Tokens with unit frequency should be modeled as "\<UNK>".

Tokenize by sentence

In [15]:
import io
import re
from typing import Iterator, Optional, List

# Regex simple para segmentar oraciones:
# divide cuando encuentra . ! ? seguidos de espacio(s)
_SENT_SPLIT: re.Pattern[str] = re.compile(r'(?<=[.!?])\s+')

# Tamaño de bloque para lectura en streaming (1 MiB)
_CHUNK_BYTES: int = 1 * 1024 * 1024


def stream_posts_from_xml(path: str) -> Iterator[str]:
    """
    Extrae contenido de <post>...</post> desde un archivo XML-like en modo streaming.

    Lee línea a línea sin cargar el archivo completo en memoria. Devuelve cada <post>
    como una cadena independiente (sin las etiquetas).

    Args:
        path (str): Ruta del archivo de entrada (XML-like, p.ej. blogs del BAC).

    Yields:
        Iterator[str]: Texto crudo de cada post (entre <post> y </post>).

    Notas:
        - Se usa encoding 'latin-1' con errors='ignore' porque los datasets suelen
          traer caracteres sueltos; evita excepciones sin romper el flujo.
        - No usa re.DOTALL para no construir una mega-cadena en memoria.
    """
    in_post: bool = False
    buf: List[str] = []

    with io.open(path, 'r', encoding='latin-1', errors='ignore') as f:
        for line in f:
            if '<post>' in line:
                in_post = True
                # Si hay contenido luego de <post> en la misma línea, lo tomamos
                line = line.split('<post>', 1)[1]

            if '</post>' in line and in_post:
                # Cerramos el post actual y emitimos
                before, _ = line.split('</post>', 1)
                buf.append(before)
                yield ''.join(buf)
                buf = []
                in_post = False
                continue

            if in_post:
                buf.append(line)


def stream_sentences(
    path: str,
    is_xml: bool = False,
    max_chars: Optional[int] = None
) -> Iterator[str]:
    """
    Tokeniza por oraciones EN STREAMING y emite cada oración al vuelo.

    Para ficheros XML-like (blogs), primero extrae <post>...</post> en streaming
    y luego segmenta por oraciones. Para texto plano, lee por bloques (1 MiB)
    y segmenta sin retener todo el documento.

    Args:
        path (str): Ruta del archivo de entrada.
        is_xml (bool, opcional): Si True, interpreta el archivo como XML-like con <post>.
        max_chars (Optional[int], opcional): Límite superior de caracteres a emitir
            (suma de longitudes de oraciones emitidas). Útil para pruebas rápidas.

    Yields:
        Iterator[str]: Oraciones tokenizadas (sin saltos de línea finales).

    Detalles:
        - La segmentación usa una regex ligera: `(?<=[.!?])\\s+`.
        - Mantiene un "carry" con el posible fragmento de oración incompleto al
          final de cada bloque, para concatenarlo con el siguiente bloque.
        - El uso de streaming mantiene el uso de RAM muy bajo (buffers pequeños).
    """
    produced: int = 0  # caracteres emitidos acumulados

    def _emit_chunks(source_iter: Iterator[str]) -> Iterator[str]:
        nonlocal produced
        carry: str = ''

        for chunk in source_iter:
            if max_chars is not None and produced >= max_chars:
                break

            carry += chunk
            parts: List[str] = _SENT_SPLIT.split(carry)
            carry = parts.pop()  # posible oración incompleta queda en carry

            for s in parts:
                s = s.strip()
                if not s:
                    continue
                produced += len(s)
                if max_chars is not None and produced > max_chars:
                    return
                yield s

        # Emitir lo que quede (última oración que no terminó en . ! ?)
        carry = carry.strip()
        if carry:
            yield carry

    if is_xml:
        # Stream de posts XML-like -> stream de oraciones
        yield from _emit_chunks(stream_posts_from_xml(path))
    else:
        # Stream de texto plano en bloques de _CHUNK_BYTES
        def file_iter() -> Iterator[str]:
            with io.open(path, 'r', encoding='latin-1', errors='ignore') as f:
                for chunk in iter(lambda: f.read(_CHUNK_BYTES), ''):
                    yield chunk

        yield from _emit_chunks(file_iter())


if __name__ == '__main__':
    # Ejemplo de uso (no materializa listas; se puede iterar directamente)
    sents_20: Iterator[str] = stream_sentences("large-files/20news.txt", is_xml=False)
    sents_blg: Iterator[str] = stream_sentences("large-files/blogs.txt", is_xml=True)

    # Muestra de las primeras N oraciones (consumirá el iterador)
    # print([next(sents_20) for _ in range(5)])
    # print([next(sents_blg) for _ in range(5)])

Normalize, but DO NOT eliminate stop words.

Replace numbers with a token named NUM.


Add sentence start and end tags "\<s>\</s>".

In [16]:
import re
from typing import Iterable, Iterator, Generator, Pattern

# --- Expresiones regulares tipadas ---
_NON_ALNUM: Pattern[str] = re.compile(r"[^a-záéíóúüñ\s0-9]")  # conserva letras minúsculas ES/EN, espacios y dígitos
_MULTI_SPACE: Pattern[str] = re.compile(r"\s+")
_NUM: Pattern[str] = re.compile(r"\d+")


def normalize_preserving_stopwords(text: str) -> str:
    """
    Normaliza una cadena SIN eliminar stopwords.

    Pasos:
      1) Convierte a minúsculas.
      2) Elimina todo carácter no alfanumérico (excepto dígitos y acentos).
      3) Colapsa espacios múltiples a un solo espacio y recorta extremos.

    Args:
        text (str): Oración cruda.

    Returns:
        str: Oración normalizada (puede quedar vacía si no quedan caracteres útiles).
    """
    s: str = text.lower()
    s = _NON_ALNUM.sub("", s)
    s = _MULTI_SPACE.sub(" ", s).strip()
    return s


def replace_numbers_with_token(text: str, token: str = "NUM") -> str:
    """
    Reemplaza cualquier secuencia de dígitos por el token indicado.

    Args:
        text (str): Oración normalizada.
        token (str, opcional): Token a usar para números. Por defecto "NUM".

    Returns:
        str: Oración con números reemplazados por el token.
    """
    return _NUM.sub(token, text)


def add_sentence_tags(text: str, start_tag: str = "<s>", end_tag: str = "</s>") -> str:
    """
    Envuelve la oración con etiquetas de inicio y fin de oración.

    Args:
        text (str): Oración final (ya normalizada y con números reemplazados).
        start_tag (str, opcional): Etiqueta de inicio. Por defecto "<s>".
        end_tag   (str, opcional): Etiqueta de fin. Por defecto "</s>".

    Returns:
        str: Cadena con las etiquetas aplicadas.
    """
    return f"{start_tag} {text} {end_tag}"


def transform_sentence_stream(sents: Iterable[str]) -> Generator[str, None, None]:
    """
    Pipeline en streaming que:
      - NORMALIZA (sin eliminar stopwords)
      - REEMPLAZA números por "NUM"
      - AGREGA etiquetas <s> y </s>

    Se implementa como generador para mantener uso de memoria bajo.

    Args:
        sents (Iterable[str]): Iterable/iterador de oraciones crudas.

    Yields:
        str: Oración transformada con formato "<s> ... </s>".
    """
    for sentence in sents:
        norm: str = normalize_preserving_stopwords(sentence)
        if not norm:              # si quedó vacía, sáltala
            continue
        with_nums: str = replace_numbers_with_token(norm, token="NUM")
        yield add_sentence_tags(with_nums, start_tag="<s>", end_tag="</s>")


# --- Ejemplo de uso con tus iteradores previos ---
if __name__ == '__main__':
  stream_20 = transform_sentence_stream(sents_20)
  stream_blg = transform_sentence_stream(sents_blg)

Tokens with unit frequency should be modeled as "\<UNK>".

(10p) Select 80% of the resulting sentences---random without replacement---to build the N-gram model and the remaining 20 for evaluation. Create the following files:

- 20N\_<group\_code>\_training (training sentences)
- 20N\_<group\_code>\_testing (testing sentences)
- BAC\_<group\_code>\_training (training sentences)
- BAC\_<group\_code>\_testing (testing sentences)

In [17]:
import os
import random
import collections
from typing import Iterator, Tuple, Dict, Optional, Set


def write_stream_to_temp(stream: Iterator[str], temp_path: str, encoding: str = "utf-8") -> collections.Counter[str]:
    """
    Vuelca un stream de oraciones a un archivo temporal y acumula frecuencias de tokens.

    Lee oración por oración en streaming (sin materializar listas) y:
      - Escribe cada oración como una línea en `temp_path`.
      - Actualiza un Counter con la frecuencia de cada token (split por espacios).

    Args:
        stream (Iterator[str]): Iterador/stream de oraciones ya preprocesadas (p. ej., "<s> ... </s>").
        temp_path (str): Ruta del archivo temporal donde se graban las oraciones.
        encoding (str): Codificación para escritura (por defecto "utf-8").

    Returns:
        collections.Counter[str]: Frecuencia total de tokens en el stream.
    """
    os.makedirs(os.path.dirname(temp_path), exist_ok=True)
    counts: collections.Counter[str] = collections.Counter()
    with open(temp_path, "w", encoding=encoding) as out:
        for sent in stream:
            out.write(sent + "\n")
            counts.update(sent.split())
    return counts


def _count_lines(path: str, encoding: str = "utf-8") -> int:
    """
    Cuenta líneas de un archivo de texto en streaming.

    Args:
        path (str): Ruta del archivo.
        encoding (str): Codificación a usar.

    Returns:
        int: Número total de líneas (oraciones) en el archivo.
    """
    n: int = 0
    with open(path, "r", encoding=encoding) as f:
        for _ in f:
            n += 1
    return n


def write_unk_and_split_exact(
    temp_path: str,
    token_counts: collections.Counter[str],
    out_prefix: str,
    group_code: str,
    train_ratio: float = 0.8,
    seed: Optional[int] = 7,
    exclude_from_unk: Optional[Set[str]] = None,
    encoding: str = "utf-8",
) -> Tuple[str, str]:
    """
    Reemplaza hapax por <UNK> y divide EXACTAMENTE en 80%/20% sin reemplazo (streaming, dos pasadas).

    Algoritmo:
      1) Se cuenta el número total de líneas N (primera pasada rápida).
      2) Se fija K = floor(N * train_ratio) (tamaño exacto del entrenamiento).
      3) Segunda pasada: para cada línea i, con `remaining_train` y `remaining_total` en cuenta,
         se asigna la línea a train con probabilidad p = remaining_train / remaining_total.
         Esto garantiza exactamente K líneas en train al finalizar (sin almacenar índices).

    Args:
        temp_path (str): Archivo temporal con una oración por línea.
        token_counts (collections.Counter[str]): Conteo total de tokens (para hapax).
        out_prefix (str): Prefijo de corpus ("20N" o "BAC").
        group_code (str): Código de grupo para nombrar archivos.
        train_ratio (float): Proporción de entrenamiento (por defecto 0.8).
        seed (Optional[int]): Semilla para aleatoriedad reproducible.
        exclude_from_unk (Optional[Set[str]]): Tokens que NO deben convertirse a <UNK> (p. ej., {"<s>", "</s>"}).
        encoding (str): Codificación de IO.

    Returns:
        Tuple[str, str]: Rutas (train_path, test_path) generadas.
    """
    rng: random.Random = random.Random(seed)
    target_dir: str = "tercer-punto"
    os.makedirs(target_dir, exist_ok=True)

    train_path: str = os.path.join(target_dir, f"{out_prefix}_{group_code}_training.txt")
    test_path: str = os.path.join(target_dir, f"{out_prefix}_{group_code}_testing.txt")

    # 1) Número total de líneas
    total_lines: int = _count_lines(temp_path, encoding=encoding)
    if total_lines == 0:
        # Genera archivos vacíos coherentes
        open(train_path, "w", encoding=encoding).close()
        open(test_path, "w", encoding=encoding).close()
        return train_path, test_path

    # 2) Tamaño exacto de train
    train_needed: int = int(total_lines * train_ratio)
    # Asegura al menos 1 y como mucho N-1 si hay al menos 2 líneas (evita extremos raros)
    if total_lines >= 2:
        train_needed = max(1, min(total_lines - 1, train_needed))

    remaining_train: int = train_needed
    remaining_total: int = total_lines

    # 3) Segunda pasada: asignación exacta sin reemplazo
    with open(temp_path, "r", encoding=encoding) as inp, \
         open(train_path, "w", encoding=encoding) as ftr, \
         open(test_path, "w", encoding=encoding) as fte:

        for raw in inp:
            line: str = raw.rstrip("\n")
            tokens: list[str] = line.split()

            # Reemplazo de hapax por <UNK>
            if exclude_from_unk is None:
                exclude_from_unk = set()
            new_tokens: list[str] = [
                (tok if (token_counts[tok] != 1 or tok in exclude_from_unk) else "<UNK>")
                for tok in tokens
            ]
            out_line: str = " ".join(new_tokens) + "\n"

            # Probabilidad adaptativa para cumplir exactamente el cupo
            p_train: float = remaining_train / remaining_total
            if rng.random() < p_train:
                ftr.write(out_line)
                remaining_train -= 1
            else:
                fte.write(out_line)

            remaining_total -= 1

    return train_path, test_path


# --------------------
# Uso de alto nivel
# --------------------
if __name__ == '__main__':
    # Asumo que ya tienes los streams de oraciones transformadas:
    #   stream_20: Iterator[str]  (20 Newsgroups)
    #   stream_blg: Iterator[str] (Blogs BAC)
    # producidos por tu pipeline previo (<s> ... </s>, NUM, etc.)

    group_code: str = "group-ansada"
    EXCLUDE: Set[str] = {"<s>", "</s>"}  # no convertir etiquetas en <UNK>

    # 20 Newsgroups
    counts_20: collections.Counter[str] = write_stream_to_temp(stream_20, "tmp/20news.sents")  # type: ignore[name-defined]
    train_20, test_20 = write_unk_and_split_exact(
        temp_path="tmp/20news.sents",
        token_counts=counts_20,
        out_prefix="20N",
        group_code=group_code,
        train_ratio=0.8,
        seed=7,
        exclude_from_unk=EXCLUDE,
    )

    # BAC Blogs
    counts_blg: collections.Counter[str] = write_stream_to_temp(stream_blg, "tmp/blogs.sents")  # type: ignore[name-defined]
    train_blg, test_blg = write_unk_and_split_exact(
        temp_path="tmp/blogs.sents",
        token_counts=counts_blg,
        out_prefix="BAC",
        group_code=group_code,
        train_ratio=0.8,
        seed=7,
        exclude_from_unk=EXCLUDE,
    )

    print(f"[OK] 20N train: {train_20}")
    print(f"[OK] 20N test : {test_20}")
    print(f"[OK] BAC train: {train_blg}")
    print(f"[OK] BAC test : {test_blg}")

[OK] 20N train: tercer-punto/20N_group-ansada_training.txt
[OK] 20N test : tercer-punto/20N_group-ansada_testing.txt
[OK] BAC train: tercer-punto/BAC_group-ansada_training.txt
[OK] BAC test : tercer-punto/BAC_group-ansada_testing.txt


(50p) Build the following N-gram models using Laplace smoothing and generate an output file for each one (you choose the output structure, but be sure to provide an appropriate Python reading method/function):

- 20N\_<group\_code>\_unigrams
- 20N\_<group\_code>\_bigrams
- 20N\_<group\_code>\_trigrams
- BAC\_<group\_code>\_unigrams
- BAC\_<group\_code>\_bigrams
- BAC\_<group\_code>\_trigrams

In [18]:
import os
import io
import re
import gzip
import json
import subprocess
from typing import Iterator, Iterable, Tuple, Dict, List, Optional

# =========================
# Paths y constantes
# =========================

MODELS_DIR: str = os.path.join("cuarto-punto", "models")
TMP_DIR: str = os.path.join(MODELS_DIR, "tmp")
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(TMP_DIR, exist_ok=True)


# =========================
# Emisión de n-gramas crudos
# =========================

def emit_ngrams_files(
    corpus_file: str,
    prefix: str,
    group_code: str,
    buf_lines: int = 5000
) -> Tuple[str, str, str]:
    """
    Emite archivos de texto con unigrama/bigrama/trigrama (uno por línea) en formato tabulado.

    Para cada oración del corpus (una por línea), escribe:
      - unigrama: 1 token por línea
      - bigrama : "w1\\tw2"
      - trigrama: "w1\\tw2\\tw3"

    Args:
        corpus_file (str): Ruta del corpus de entrenamiento (una oración por línea).
        prefix (str): Prefijo del corpus ("20N" o "BAC").
        group_code (str): Código de grupo.
        buf_lines (int): Tamaño de buffer para volcar a disco en lotes.

    Returns:
        Tuple[str, str, str]: Rutas de salida (uni_path, bi_path, tri_path).
    """
    uni_path: str = os.path.join(TMP_DIR, f"{prefix}_{group_code}.uni.txt")
    bi_path: str = os.path.join(TMP_DIR, f"{prefix}_{group_code}.bi.txt")
    tri_path: str = os.path.join(TMP_DIR, f"{prefix}_{group_code}.tri.txt")

    with open(corpus_file, "r", encoding="utf-8") as f, \
         open(uni_path, "w", encoding="utf-8") as fu, \
         open(bi_path, "w", encoding="utf-8") as fb, \
         open(tri_path, "w", encoding="utf-8") as ft:

        bu: List[str] = []
        bb: List[str] = []
        bt: List[str] = []

        for line in f:
            tok: List[str] = line.strip().split()
            n: int = len(tok)
            if n == 0:
                continue

            # unigrams
            bu.extend(tok)
            # bigrams
            for i in range(n - 1):
                bb.append(f"{tok[i]}\t{tok[i+1]}")
            # trigrams
            for i in range(n - 2):
                bt.append(f"{tok[i]}\t{tok[i+1]}\t{tok[i+2]}")

            # volcados por lote
            if len(bu) >= buf_lines:
                fu.write("\n".join(bu) + "\n"); bu = []
            if len(bb) >= buf_lines:
                fb.write("\n".join(bb) + "\n"); bb = []
            if len(bt) >= buf_lines:
                ft.write("\n".join(bt) + "\n"); bt = []

        # resto
        if bu: fu.write("\n".join(bu) + "\n")
        if bb: fb.write("\n".join(bb) + "\n")
        if bt: ft.write("\n".join(bt) + "\n")

    return uni_path, bi_path, tri_path


def sort_count(in_path: str) -> str:
    """
    Ordena y cuenta líneas con herramientas del sistema (sort + uniq -c).
    Usa external merge sort (RAM baja y muy rápido).

    Args:
        in_path (str): Archivo de entrada (tokens por línea).

    Returns:
        str: Ruta del archivo de salida con conteos (formato 'COUNT<space>KEY').
    """
    out_path: str = in_path + ".counts"
    subprocess.run(
        ["bash", "-lc", f"LC_ALL=C sort {in_path} | uniq -c > {out_path}"],
        check=True
    )
    return out_path


def stream_counts(fp: io.TextIOBase) -> Iterator[Tuple[int, str]]:
    """
    Itera líneas de un archivo 'uniq -c' del tipo: '   COUNT KEY'.

    Args:
        fp (io.TextIOBase): Archivo abierto en modo texto.

    Yields:
        Iterator[Tuple[int, str]]: Pares (count, key) por línea.
    """
    for line in fp:
        line = line.rstrip("\n")
        if not line:
            continue
        i: int = 0
        # espacios previos
        while i < len(line) and line[i] == ' ':
            i += 1
        j: int = i
        while j < len(line) and line[j].isdigit():
            j += 1
        cnt: int = int(line[i:j])
        key: str = line[j+1:]  # salta el espacio separador tras el conteo
        yield cnt, key


# =========================
# Escritura de modelos (Laplace)
# =========================

def write_models_from_counts(
    prefix: str,
    group_code: str,
    uni_counts_path: str,
    bi_counts_path: str,
    tri_counts_path: str
) -> Tuple[str, str, str]:
    """
    Construye modelos n-gram Laplace y los escribe a JSON.GZ en streaming.

    Estructura de salida (JSON dict):
      - unigrams: {"w": p, ...}
      - bigrams : {"w1 w2": p, ...}
      - trigrams: {"w1 w2 w3": p, ...}

    Probabilidades (Laplace):
      - Unigrama: p = (c + 1) / (N + V)
      - Bigrama : p = (c + 1) / (sum_{w2} c(w1,w2) + V)
      - Trigrama: p = (c + 1) / (sum_{w3} c(w1,w2,w3) + V)

    Args:
        prefix (str): "20N" o "BAC".
        group_code (str): Código de grupo.
        uni_counts_path (str): Ruta de conteos de unigramas.
        bi_counts_path (str): Ruta de conteos de bigramas.
        tri_counts_path (str): Ruta de conteos de trigramas.

    Returns:
        Tuple[str, str, str]: Rutas (unigrams_json_gz, bigrams_json_gz, trigrams_json_gz).
    """
    # ---- 1) Unigramas: total tokens (N) y vocab (V) ----
    total_tokens: int = 0
    vocab_size: int = 0
    with open(uni_counts_path, "r", encoding="utf-8") as f:
        for c, _w in stream_counts(f):
            total_tokens += c
            vocab_size += 1
    denom_uni: int = total_tokens + vocab_size

    # ---- 2) Unigram model JSON.gz ----
    uni_json: str = os.path.join(MODELS_DIR, f"{prefix}_{group_code}_unigrams.json.gz")
    with gzip.open(uni_json, "wt", encoding="utf-8") as out, \
         open(uni_counts_path, "r", encoding="utf-8") as f:
        out.write("{")
        first: bool = True
        for c, w in stream_counts(f):
            p: float = (c + 1) / denom_uni
            if not first: out.write(",")
            first = False
            out.write(f'\n"{w}": {p}')
        out.write("\n}")
    print(f"[OK] {uni_json}")

    # ---- 3) Bigramas: ctx(w1) = sum_{w2} c(w1,w2) ----
    bi_json: str = os.path.join(MODELS_DIR, f"{prefix}_{group_code}_bigrams.json.gz")
    with gzip.open(bi_json, "wt", encoding="utf-8") as out, \
         open(bi_counts_path, "r", encoding="utf-8") as f:
        out.write("{")
        first = True
        curr_w1: Optional[str] = None
        buffer: List[Tuple[str, str, int]] = []
        ctx_sum: int = 0

        def flush_group() -> None:
            nonlocal first
            if curr_w1 is None:
                return
            denom_ctx: int = ctx_sum + vocab_size
            for (w1, w2, c) in buffer:
                p = (c + 1) / denom_ctx
                if not first: out.write(",")
                first = False
                out.write(f'\n"{w1} {w2}": {p}')

        for c, key in stream_counts(f):
            w1, w2 = key.split("\t", 1)
            if curr_w1 is None:
                curr_w1 = w1
            if w1 != curr_w1:
                flush_group()
                buffer.clear()
                ctx_sum = 0
                curr_w1 = w1
            buffer.append((w1, w2, c))
            ctx_sum += c

        flush_group()
        out.write("\n}")
    print(f"[OK] {bi_json}")

    # ---- 4) Trigramas: ctx(w1,w2) = sum_{w3} c(w1,w2,w3) ----
    tri_json: str = os.path.join(MODELS_DIR, f"{prefix}_{group_code}_trigrams.json.gz")
    with gzip.open(tri_json, "wt", encoding="utf-8") as out, \
         open(tri_counts_path, "r", encoding="utf-8") as f:
        out.write("{")
        first = True
        curr_ctx: Optional[str] = None  # "w1\\tw2"
        buffer_t: List[Tuple[str, str, str, int]] = []
        ctx_sum_t: int = 0

        def flush_group_tri() -> None:
            nonlocal first
            if curr_ctx is None:
                return
            denom_ctx: int = ctx_sum_t + vocab_size
            for (w1, w2, w3, c) in buffer_t:
                p = (c + 1) / denom_ctx
                if not first: out.write(",")
                first = False
                out.write(f'\n"{w1} {w2} {w3}": {p}')

        for c, key in stream_counts(f):
            parts: List[str] = key.split("\t")
            if len(parts) != 3:
                continue
            w1, w2, w3 = parts
            ctx: str = f"{w1}\t{w2}"
            if curr_ctx is None:
                curr_ctx = ctx
            if ctx != curr_ctx:
                flush_group_tri()
                buffer_t.clear()
                ctx_sum_t = 0
                curr_ctx = ctx
            buffer_t.append((w1, w2, w3, c))
            ctx_sum_t += c

        flush_group_tri()
        out.write("\n}")
    print(f"[OK] {tri_json}")
    print(f"[DONE] {prefix} vocab={vocab_size} tokens={total_tokens}")

    return uni_json, bi_json, tri_json


def build_ngram_models_external_sort(
    corpus_file: str,
    group_code: str,
    file_prefix: str
) -> Tuple[str, str, str]:
    """
    Pipeline completo: emitir n-gramas → ordenar/contar → escribir modelos JSON.gz (Laplace).

    Args:
        corpus_file (str): Corpus de entrenamiento (una oración por línea).
        group_code (str): Código de grupo.
        file_prefix (str): "20N" o "BAC".

    Returns:
        Tuple[str, str, str]: Rutas JSON.gz (uni, bi, tri) generadas.
    """
    uni_txt, bi_txt, tri_txt = emit_ngrams_files(corpus_file, file_prefix, group_code)
    uni_cnt: str = sort_count(uni_txt)
    bi_cnt: str = sort_count(bi_txt)
    tri_cnt: str = sort_count(tri_txt)
    paths = write_models_from_counts(file_prefix, group_code, uni_cnt, bi_cnt, tri_cnt)
    # Limpieza opcional
    for p in [uni_txt, bi_txt, tri_txt, uni_cnt, bi_cnt, tri_cnt]:
        try:
            os.remove(p)
        except OSError:
            pass
    return paths


# =========================
# Lectores de modelos
# =========================

def load_ngram_model(prefix: str, group_code: str, n: int) -> Dict[str, float]:
    """
    Carga un modelo n-grama completo a memoria desde JSON(.gz).

    Args:
        prefix (str): "20N" o "BAC".
        group_code (str): Código de grupo.
        n (int): Orden del n-grama (1, 2 o 3).

    Returns:
        Dict[str, float]: Diccionario {clave: prob}. Clave:
            - n=1: "w"
            - n=2: "w1 w2"
            - n=3: "w1 w2 w3"
    """
    suffix_map: Dict[int, str] = {1: "unigrams", 2: "bigrams", 3: "trigrams"}
    base: str = os.path.join(MODELS_DIR, f"{prefix}_{group_code}_{suffix_map[n]}")
    path_gz: str = base + ".json.gz"
    path_json: str = base + ".json"

    path: Optional[str] = None
    if os.path.exists(path_gz):
        path = path_gz
    elif os.path.exists(path_json):
        path = path_json
    else:
        raise FileNotFoundError(f"No se encontró modelo: {path_gz} ni {path_json}")

    if path.endswith(".gz"):
        with gzip.open(path, "rt", encoding="utf-8") as f:
            return json.load(f)
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def iter_ngram_model_stream(prefix: str, group_code: str, n: int) -> Iterator[Tuple[str, float]]:
    """
    Iterador que lee el JSON(.gz) del modelo línea a línea (streaming).
    Útil para trigramas grandes (evita picos de RAM).

    Args:
        prefix (str): "20N" o "BAC".
        group_code (str): Código de grupo.
        n (int): Orden del n-grama (1, 2 o 3).

    Yields:
        Iterator[Tuple[str, float]]: Pares (clave, prob).
    """
    suffix_map: Dict[int, str] = {1: "unigrams", 2: "bigrams", 3: "trigrams"}
    base: str = os.path.join(MODELS_DIR, f"{prefix}_{group_code}_{suffix_map[n]}")
    path_gz: str = base + ".json.gz"
    path_json: str = base + ".json"
    path: str
    if os.path.exists(path_gz):
        path = path_gz
    elif os.path.exists(path_json):
        path = path_json
    else:
        raise FileNotFoundError(f"No se encontró modelo: {path_gz} ni {path_json}")

    pat: re.Pattern[str] = re.compile(r'^\s*"([^"]+)":\s*([0-9eE+\-\.]+)\s*,?\s*$')
    opener = gzip.open if path.endswith(".gz") else open
    with opener(path, "rt", encoding="utf-8") as f:
        _ = f.readline()  # '{'
        for line in f:
            line = line.strip()
            if line == "}":
                break
            m = pat.match(line)
            if not m:
                continue
            key: str = m.group(1)
            p: float = float(m.group(2))
            yield key, p


# =========================
# Ejemplo de construcción
# =========================
if __name__ == "__main__":
    # --- Rutas de entrenamiento generadas en el paso 80/20 ---
    train_20 = os.path.join("tercer-punto", f"20N_{group_code}_training.txt")
    train_blg = os.path.join("tercer-punto", f"BAC_{group_code}_training.txt")

    # --- Validaciones rápidas ---
    for p in (train_20, train_blg):
        if not os.path.isfile(p):
            raise FileNotFoundError(f"Falta el corpus de entrenamiento: {p}")

    # --- Construcción de modelos con Laplace (external sort; RAM baja) ---
    u20, b20, t20 = build_ngram_models_external_sort(train_20, group_code, "20N")
    uB,  bB,  tB  = build_ngram_models_external_sort(train_blg, group_code, "BAC")

    print("[OK] 20N ->", u20, b20, t20)
    print("[OK] BAC ->", uB,  bB,  tB)

    # --- Lectura de ejemplo ---
    # Unigramas/bigramas: carga completa (son manejables)
    uni20 = load_ngram_model("20N", group_code, 1)
    print("Ejemplo P('the') en 20N:", uni20.get("the", None))

    # Trigramas grandes (p.ej. BAC): streaming sin cargar en RAM
    print("Primeros 3 trigramas BAC (streaming):")
    for i, (k, p) in zip(range(3), iter_ngram_model_stream("BAC", group_code, 3)):
        print("  ", k, "->", p)

[OK] cuarto-punto/models/20N_group-ansada_unigrams.json.gz
[OK] cuarto-punto/models/20N_group-ansada_bigrams.json.gz
[OK] cuarto-punto/models/20N_group-ansada_trigrams.json.gz
[DONE] 20N vocab=74376 tokens=4415056
[OK] cuarto-punto/models/BAC_group-ansada_unigrams.json.gz
[OK] cuarto-punto/models/BAC_group-ansada_bigrams.json.gz
[OK] cuarto-punto/models/BAC_group-ansada_trigrams.json.gz
[DONE] BAC vocab=457103 tokens=123531284
[OK] 20N -> cuarto-punto/models/20N_group-ansada_unigrams.json.gz cuarto-punto/models/20N_group-ansada_bigrams.json.gz cuarto-punto/models/20N_group-ansada_trigrams.json.gz
[OK] BAC -> cuarto-punto/models/BAC_group-ansada_unigrams.json.gz cuarto-punto/models/BAC_group-ansada_bigrams.json.gz cuarto-punto/models/BAC_group-ansada_trigrams.json.gz
Ejemplo P('the') en 20N: 0.04244256288991569
Primeros 3 trigramas BAC (streaming):
   <UNK> <UNK> </s> -> 0.006131926872148207
   <UNK> <UNK> <UNK> -> 0.016192595754194485
   <UNK> <UNK> NUM -> 0.0008251780941637958


(15p) Using the test dataset, calculate the perplexity of each language model. Report the results obtained. If you experience variable overflow, use probabilities in log space.

El link de la carpeta donde estan los conjuntos de entrenamiento + testing y los modelos entrenados es:

https://uniandes-my.sharepoint.com/:f:/g/personal/a_mosquerah2_uniandes_edu_co/Em-od1gldI9BnnTjpUXqZXcB8fjXUoybI35zktWPWzyqpw?e=6dT4ng

(Solo se puede abrir con correo uniandes)

In [ ]:
# carpeta conjunto de entrenamiento y de testing de 20news y de BAC
tercer_punto_folder = "tercer-punto"
# carpeta modelos de unigramas, bigramas y trigramas de cada corpus.
cuarto_punto_folder = "cuarto-punto"

group_code = "group-ansada"

In [20]:
# =========================
# Perplejidad con backoff y trigramas en DISCO (SQLite) — Tipado + Docstrings
# =========================
import os
import re
import gzip
import json
import math
import sqlite3
from functools import lru_cache
from typing import Dict, Optional, Tuple, Callable, Iterator

tercer_punto_folder: str = "tercer-punto"
cuarto_punto_folder: str = "cuarto-punto"
group_code: str = "group-ansada"


def _json_load_any(path: str) -> Dict[str, float]:
    """
    Carga un diccionario JSON {clave: prob} desde .json o .json.gz sin materializar más de lo necesario.

    Args:
        path: Ruta al archivo .json o .json.gz.

    Returns:
        Dict[str, float]: Mapa clave→probabilidad (por ejemplo, "w1 w2": 0.00123).
    """
    if path.endswith(".gz"):
        with gzip.open(path, "rt", encoding="utf-8") as f:  # type: ignore[name-defined]
            return json.load(f)
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def read_ngram_model_smart(prefix: str, n: int) -> Optional[Dict[str, float]]:
    """
    Localiza y carga el modelo n-grama del corpus dado.

    Convención de nombres:
      n=1 -> *_unigrams.json(.gz)
      n=2 -> *_bigrams.json(.gz)
      n=3 -> *_trigrams.json(.gz)

    Args:
        prefix: "20N" o "BAC".
        n: Orden del modelo (1, 2, 3).

    Returns:
        Dict[str, float] si existe; None en caso contrario.
    """
    suffix: str = {1: "uni", 2: "bi", 3: "tri"}[n]
    base: str = os.path.join(cuarto_punto_folder, "models", f"{prefix}_{group_code}")
    candidates = [f"{base}_{suffix}grams.json", f"{base}_{suffix}grams.json.gz"]
    for p in candidates:
        if os.path.exists(p):
            try:
                model = _json_load_any(p)
                print(f"[OK] Modelo {n}-gramas cargado: {os.path.basename(p)}")
                return model
            except Exception as e:  # pragma: no cover (logging)
                print(f"[WARN] Falló carga de {p}: {e}")
    print(f"[ERROR] No se encontró modelo {n}-gramas para {prefix}")
    return None


def _trigram_json_gz_to_sqlite(gz_path: str, sqlite_path: str) -> None:
    """
    Convierte un JSON(.gz) de trigramas {"w1 w2 w3": p, ...} a una tabla SQLite:
        tri(w1 TEXT, w2 TEXT, w3 TEXT, p REAL, PRIMARY KEY(w1,w2,w3))

    Se procesa en streaming para evitar usar RAM alta.

    Args:
        gz_path: Ruta al .json.gz de trigramas.
        sqlite_path: Ruta de salida .sqlite.
    """
    if os.path.exists(sqlite_path):
        return
    os.makedirs(os.path.dirname(sqlite_path), exist_ok=True)
    conn = sqlite3.connect(sqlite_path)
    cur = conn.cursor()
    cur.executescript(
        "PRAGMA journal_mode=OFF; PRAGMA synchronous=OFF; PRAGMA temp_store=MEMORY;"
        "CREATE TABLE tri (w1 TEXT, w2 TEXT, w3 TEXT, p REAL, PRIMARY KEY(w1,w2,w3));"
    )

    pat: re.Pattern[str] = re.compile(r'^\s*"([^"]+)":\s*([0-9eE+\-\.]+)\s*,?\s*$')
    batch: list[tuple[str, str, str, float]] = []

    with gzip.open(gz_path, "rt", encoding="utf-8") as f:  # type: ignore[name-defined]
        _ = f.readline()  # '{'
        for line in f:
            s = line.strip()
            if s == "}":
                break
            m = pat.match(s)
            if not m:
                continue
            key, p_str = m.group(1), m.group(2)
            parts = key.split(" ")
            if len(parts) != 3:
                continue
            batch.append((parts[0], parts[1], parts[2], float(p_str)))
            if len(batch) >= 50_000:
                cur.executemany("INSERT OR REPLACE INTO tri(w1,w2,w3,p) VALUES(?,?,?,?)", batch)
                conn.commit()
                batch.clear()
    if batch:
        cur.executemany("INSERT OR REPLACE INTO tri(w1,w2,w3,p) VALUES(?,?,?,?)", batch)
        conn.commit()
    cur.execute("ANALYZE")
    conn.commit()
    conn.close()
    print(f"[OK] Trigramas indexados en {sqlite_path}")


def _make_tri_getter(prefix: str) -> Tuple[Callable[[str, str, str], Optional[float]], sqlite3.Connection]:
    """
    Prepara el acceso a trigramas vía SQLite y devuelve:
      - una función tri_get(w1, w2, w3) -> prob o None
      - la conexión SQLite (para cerrar al terminar)

    Args:
        prefix: "20N" o "BAC".

    Returns:
        (tri_get, conn)
    """
    gz = os.path.join(cuarto_punto_folder, "models", f"{prefix}_{group_code}_trigrams.json.gz")
    db = os.path.join(cuarto_punto_folder, "models", f"{prefix}_{group_code}_trigrams.sqlite")
    if not os.path.exists(gz):
        gz_alt = gz[:-3]
        if os.path.exists(gz_alt):
            raise RuntimeError("Convierte el .json de trigramas a .json.gz o ajusta el parser.")
        raise FileNotFoundError(f"No existe {gz} ni {gz_alt}")
    _trigram_json_gz_to_sqlite(gz, db)

    conn = sqlite3.connect(db)
    cur = conn.cursor()

    @lru_cache(maxsize=200_000)
    def tri_get(a: str, b: str, c: str) -> Optional[float]:
        row = cur.execute("SELECT p FROM tri WHERE w1=? AND w2=? AND w3=?", (a, b, c)).fetchone()
        return float(row[0]) if row else None

    return tri_get, conn


def calculate_perplexity_backoff_sqltri(
    testing_file: str,
    uni: Dict[str, float],
    bi: Dict[str, float],
    tri_get: Callable[[str, str, str], Optional[float]]
) -> Tuple[float, float, float]:
    """
    Calcula perplejidad de modelos unigrama/bigrama/trigrama sobre un corpus de prueba.

    - Unigrama: usa p_unigram(w). Si falta, usa epsilon.
    - Bigrama : P(w1) * Π P(w_i | w_{i-1}); si (w_{i-1}, w_i) falta, backoff a p_unigram(w_i).
    - Trigrama: P(w1) * P(w2|w1) * Π P(w_i | w_{i-2}, w_{i-1});
                si tri falta -> backoff a bi; si bi falta -> backoff a uni.

    Se acumulan log-probabilidades con *clipping* para evitar underflow.

    Args:
        testing_file: Ruta del archivo de prueba (una oración por línea).
        uni: Diccionario unigrama {"w": p}.
        bi: Diccionario bigrama {"w1 w2": p}.
        tri_get: Función que consulta trigramas en disco.

    Returns:
        (ppl_unigram, ppl_bigram, ppl_trigram)
    """
    epsilon_uni: float = 1.0 / max(1, 10 * len(uni))  # piso seguro OOV
    total_log_u: float = 0.0
    total_log_b: float = 0.0
    total_log_t: float = 0.0
    total_tokens: int = 0

    def _log(p: float) -> float:
        return math.log(p if p > 1e-300 else 1e-300)

    with open(testing_file, "r", encoding="utf-8") as f:
        for raw in f:
            s = raw.strip()
            if not s:
                continue
            toks = s.split()
            n = len(toks)
            if n == 0:
                continue

            total_tokens += n

            # --- Unigram ---
            for w in toks:
                p = uni.get(w, epsilon_uni)
                total_log_u += _log(p)

            # --- Bigram ---
            p1 = uni.get(toks[0], epsilon_uni)
            total_log_b += _log(p1)
            for i in range(n - 1):
                a, b_ = toks[i], toks[i + 1]
                p_bi = bi.get(f"{a} {b_}")
                if p_bi is None:
                    p_bi = uni.get(b_, epsilon_uni)
                total_log_b += _log(p_bi)

            # --- Trigram ---
            p1 = uni.get(toks[0], epsilon_uni)
            total_log_t += _log(p1)
            if n >= 2:
                p2 = bi.get(f"{toks[0]} {toks[1]}")
                if p2 is None:
                    p2 = uni.get(toks[1], epsilon_uni)
                total_log_t += _log(p2)
            for i in range(n - 2):
                a, b_, c = toks[i], toks[i + 1], toks[i + 2]
                p_tri = tri_get(a, b_, c)
                if p_tri is None:
                    p_tri = bi.get(f"{b_} {c}")
                    if p_tri is None:
                        p_tri = uni.get(c, epsilon_uni)
                total_log_t += _log(p_tri)

    if total_tokens == 0:
        return float("inf"), float("inf"), float("inf")

    ppl_u: float = math.exp(-total_log_u / total_tokens)
    ppl_b: float = math.exp(-total_log_b / total_tokens)
    ppl_t: float = math.exp(-total_log_t / total_tokens)
    return ppl_u, ppl_b, ppl_t


def run_perplexity_for(prefix: str) -> Optional[Tuple[float, float, float]]:
    """
    Ejecuta el cálculo de perplejidad para un corpus ('20N' o 'BAC'):
      - Carga uni/bi en RAM.
      - Abre acceso a tri en SQLite.
      - Evalúa sobre el archivo de testing de 'tercer-punto/'.

    Args:
        prefix: "20N" o "BAC".

    Returns:
        (ppl_uni, ppl_bi, ppl_tri) o None si faltan modelos.
    """
    testing_file: str = os.path.join(tercer_punto_folder, f"{prefix}_{group_code}_testing.txt")

    uni_raw = read_ngram_model_smart(prefix, 1)
    bi_raw = read_ngram_model_smart(prefix, 2)
    if not all([uni_raw, bi_raw]):
        print(f"[SKIP] Faltan uni/bi para {prefix}")
        return None

    # Llaves en RAM tal como están en archivo (sin duplicar estructuras)
    uni: Dict[str, float] = {k: float(v) for k, v in uni_raw.items()}
    bi: Dict[str, float] = {k: float(v) for k, v in bi_raw.items()}

    tri_get, conn = _make_tri_getter(prefix)
    try:
        pu, pb, pt = calculate_perplexity_backoff_sqltri(testing_file, uni, bi, tri_get)
        print(f"\n=== Perplejidad {prefix} ===")
        print(f"Unigramas : {pu:.4f}")
        print(f"Bigramas  : {pb:.4f}")
        print(f"Trigramas : {pt:.4f}")
        return pu, pb, pt
    finally:
        conn.close()


def run_all_and_report(save_path: Optional[str] = None) -> Dict[str, Dict[str, float]]:
    """
    Ejecuta perplejidad para ambos corpus (20N/BAC) y devuelve (y opcionalmente guarda) un reporte.

    Args:
        save_path: Ruta JSON para guardar resultados; si None, no guarda.

    Returns:
        Dict con estructura:
        {
          "20N": {"unigram": ..., "bigram": ..., "trigram": ...},
          "BAC": {"unigram": ..., "bigram": ..., "trigram": ...}
        }
    """
    report: Dict[str, Dict[str, float]] = {}
    for prefix in ("20N", "BAC"):
        res = run_perplexity_for(prefix)
        if res is not None:
            pu, pb, pt = res
            report[prefix] = {"unigram": pu, "bigram": pb, "trigram": pt}
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(report, f, ensure_ascii=False, indent=2)
        print(f"[OK] Reporte guardado en {save_path}")
    return report


if __name__ == '__main__':
    print("\n--- Calculando perplejidad (RAM<10GB, VRAM≈0, trigram en SQLite) ---")
    run_perplexity_for("20N")
    run_perplexity_for("BAC")
    # Opcional: guardar reporte
    # run_all_and_report(os.path.join(cuarto_punto_folder, "perplexity_report.json"))


--- Calculando perplejidad (RAM<10GB, VRAM≈0, trigram en SQLite) ---
[OK] Modelo 1-gramas cargado: 20N_group-ansada_unigrams.json.gz
[OK] Modelo 2-gramas cargado: 20N_group-ansada_bigrams.json.gz
[OK] Trigramas indexados en cuarto-punto/models/20N_group-ansada_trigrams.sqlite

=== Perplejidad 20N ===
Unigramas : 1097.4650
Bigramas  : 1878.5108
Trigramas : 4831.0297
[OK] Modelo 1-gramas cargado: BAC_group-ansada_unigrams.json.gz
[OK] Modelo 2-gramas cargado: BAC_group-ansada_bigrams.json.gz
[OK] Trigramas indexados en cuarto-punto/models/BAC_group-ansada_trigrams.sqlite

=== Perplejidad BAC ===
Unigramas : 828.9353
Bigramas  : 1041.2564
Trigramas : 5463.5486


(15p) Using your best language model, build a method/function that automatically generates sentences by receiving the first word of a sentence as input. Take different tests and document them.

In [22]:
# ===============================
# Generador de oraciones (RAM<10GB) usando trigramas en SQLite + backoff
# ===============================
import os
import re
import json
import gzip
import math
import sqlite3
import random
from functools import lru_cache
from typing import Dict, List, Tuple, Optional, Callable

cuarto_punto_folder: str = "cuarto-punto"
group_code: str = "group-ansada"


# ---------- Utils ----------
def _json_load_any(path: str) -> Dict[str, float]:
    """
    Carga un diccionario JSON {clave: prob} desde .json o .json.gz.

    Args:
        path: Ruta del archivo.

    Returns:
        Dict[str, float]: Mapa de probabilidades.
    """
    if path.endswith(".gz"):
        with gzip.open(path, "rt", encoding="utf-8") as f:
            return json.load(f)
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def _path(prefix: str, kind: str) -> str:
    """
    Construye rutas a modelos.

    Args:
        prefix: "20N" o "BAC".
        kind: uno de {"uni","bi","tri_json","tri_db"}.

    Returns:
        str: Ruta al recurso solicitado.
    """
    base = os.path.join(cuarto_punto_folder, "models", f"{prefix}_{group_code}")
    if kind == "uni":
        return base + "_unigrams.json.gz" if os.path.exists(base + "_unigrams.json.gz") else base + "_unigrams.json"
    if kind == "bi":
        return base + "_bigrams.json.gz" if os.path.exists(base + "_bigrams.json.gz") else base + "_bigrams.json"
    if kind == "tri_json":
        return base + "_trigrams.json.gz" if os.path.exists(base + "_trigrams.json.gz") else base + "_trigrams.json"
    if kind == "tri_db":
        return base + "_trigrams.sqlite"
    raise ValueError(f"kind inválido: {kind}")


# ---------- JSON(.gz) -> SQLite para trigramas (streaming) ----------
def _trigram_json_to_sqlite(tri_json_path: str, tri_db_path: str) -> None:
    """
    Indexa trigramas {"w1 w2 w3": p} en SQLite (tabla tri(w1,w2,w3,p)) sin cargar todo en RAM.

    Args:
        tri_json_path: Ruta a *_trigrams.json(.gz).
        tri_db_path: Ruta de salida *.sqlite.
    """
    if os.path.exists(tri_db_path):
        return
    os.makedirs(os.path.dirname(tri_db_path), exist_ok=True)
    conn = sqlite3.connect(tri_db_path)
    cur = conn.cursor()
    cur.executescript("""
        PRAGMA journal_mode=OFF;
        PRAGMA synchronous=OFF;
        PRAGMA temp_store=MEMORY;
        CREATE TABLE tri (w1 TEXT, w2 TEXT, w3 TEXT, p REAL, PRIMARY KEY(w1,w2,w3));
    """)

    pat: re.Pattern[str] = re.compile(r'^\s*"([^"]+)":\s*([0-9eE+\-\.]+)\s*,?\s*$')
    batch: List[Tuple[str, str, str, float]] = []

    fh = gzip.open(tri_json_path, "rt", encoding="utf-8") if tri_json_path.endswith(".gz") \
         else open(tri_json_path, "r", encoding="utf-8")
    with fh:
        _ = fh.readline()  # '{'
        for line in fh:
            s = line.strip()
            if s == "}":
                break
            m = pat.match(s)
            if not m:
                continue
            key, p_str = m.group(1), m.group(2)
            parts = key.split(" ")
            if len(parts) != 3:
                continue
            batch.append((parts[0], parts[1], parts[2], float(p_str)))
            if len(batch) >= 50_000:
                cur.executemany("INSERT OR REPLACE INTO tri(w1,w2,w3,p) VALUES(?,?,?,?)", batch)
                conn.commit()
                batch.clear()
    if batch:
        cur.executemany("INSERT OR REPLACE INTO tri(w1,w2,w3,p) VALUES(?,?,?,?)", batch)
        conn.commit()

    cur.execute("ANALYZE")
    conn.commit()
    conn.close()
    print(f"[OK] Trigramas indexados: {tri_db_path}")


def _make_tri_access(prefix: str) -> Tuple[Callable[[str, str], List[Tuple[str, float]]], sqlite3.Connection]:
    """
    Prepara acceso rápido a seguidores de trigramas vía SQLite.

    Args:
        prefix: "20N" o "BAC".

    Returns:
        (followers_fn, conn) donde followers_fn(w1,w2) -> [(w3, p), ...] ordenado por p desc.
    """
    tri_json = _path(prefix, "tri_json")
    tri_db = _path(prefix, "tri_db")
    _trigram_json_to_sqlite(tri_json, tri_db)
    conn = sqlite3.connect(tri_db)
    cur = conn.cursor()

    @lru_cache(maxsize=200_000)
    def followers(w1: str, w2: str) -> List[Tuple[str, float]]:
        rows = cur.execute("SELECT w3,p FROM tri WHERE w1=? AND w2=?", (w1, w2)).fetchall()
        rows.sort(key=lambda x: x[1], reverse=True)
        return [(str(w3), float(p)) for (w3, p) in rows]

    return followers, conn


# ---------- Muestreador ----------
def _sample(
    scored: List[Tuple[str, float]],
    temperature: float = 0.9,
    top_k: int = 50,
    top_p: float = 0.95,
    rng: Optional[random.Random] = None
) -> str:
    """
    Muestrea un token desde una lista (token, score) con temperatura + top-k + nucleus (top-p).

    Args:
        scored: Candidatos (token, puntuación/probabilidad).
        temperature: Suavizado (>=0). 1.0 = sin cambio.
        top_k: recorta a los k mejores (si >0).
        top_p: recorta por prob acumulada (0<p<=1).
        rng: generador aleatorio (reproducible si se pasa semilla).

    Returns:
        str: token elegido (o </s> si no hay candidatos).
    """
    if not scored:
        return "</s>"
    if rng is None:
        rng = random

    # Orden y top-k
    scored = sorted(scored, key=lambda x: x[1], reverse=True)
    if top_k and top_k > 0:
        scored = scored[:min(top_k, len(scored))]

    # Normaliza a prob y aplica top-p (nucleus)
    total = sum(max(0.0, p) for _, p in scored) or 1e-12
    probs = [max(0.0, p) / total for _, p in scored]

    cut: List[Tuple[str, float]] = []
    acc = 0.0
    for (tok, _), p in zip(scored, probs):
        cut.append((tok, p))
        acc += p
        if top_p is not None and acc >= top_p:
            break

    # Temperatura sobre log-prob
    logs = [math.log(max(1e-12, p)) / max(1e-6, temperature) for _, p in cut]
    m = max(logs)
    exps = [math.exp(x - m) for x in logs]
    z = sum(exps)
    adj = [e / z for e in exps]

    r = rng.random()
    c = 0.0
    for (tok, _), p in zip(cut, adj):
        c += p
        if r <= c:
            return tok
    return cut[-1][0]


# ---------- Generador (elige orden 1/2/3; por defecto 3 con backoff) ----------
def build_sentence_generator(prefix: str = "20N", default_seed: Optional[int] = 7):
    """
    Construye un generador de oraciones para el corpus dado.

    Carga unigrama y bigrama en RAM; trigramas se consultan en SQLite.
    Por defecto genera con TRIGRAMAS (mejor fluidez) y backoff a uni.

    Args:
        prefix: "20N" o "BAC".
        default_seed: Semilla por defecto para reproducibilidad.

    Returns:
        (gen_fn, close_fn)
          - gen_fn(first_word, max_len=30, temperature=0.9, top_k=60, top_p=0.95,
                  seed=None, order=3) -> (sentence, tokens)
          - close_fn(): cierra la conexión SQLite interna.
    """
    uni: Dict[str, float] = _json_load_any(_path(prefix, "uni"))      # {"w": p}
    bi: Dict[str, float] = _json_load_any(_path(prefix, "bi"))        # {"w1 w2": p}
    tri_followers, tri_conn = _make_tri_access(prefix)

    uni_items: List[Tuple[str, float]] = [(w, float(p)) for w, p in uni.items()]

    def gen(
        first_word: str,
        max_len: int = 30,
        temperature: float = 0.9,
        top_k: int = 60,
        top_p: float = 0.95,
        seed: Optional[int] = default_seed,
        order: int = 3
    ) -> Tuple[str, List[str]]:
        """
        Genera una oración dada la primera palabra.

        Args:
            first_word: Primera palabra de la oración (se normaliza a minúsculas).
            max_len: longitud máxima de tokens (incluye etiquetas).
            temperature: parámetro de muestreo.
            top_k: recorte por k mejores candidatos.
            top_p: nucleus sampling.
            seed: semilla para reproducibilidad.
            order: 1=unigrama (best-by-perplexity), 2=bigrama, 3=trigrama con backoff.

        Returns:
            (superficie_sin_etiquetas, lista_de_tokens_incluyendo_<s>/</s>)
        """
        rng = random.Random(seed)
        fw = first_word.strip().lower()
        if fw not in uni:
            fw = "<UNK>" if "<UNK>" in uni else fw

        tokens: List[str] = ["<s>", fw]

        def _next_from_bigram(a: str) -> str:
            # candidatos del bigrama a->?
            cands = [(w2, float(p)) for (w12, p) in bi.items() if w12.startswith(a + " ")]
            return _sample(cands if cands else uni_items, temperature, top_k, top_p, rng)

        while len(tokens) < max_len:
            if tokens[-1] == "</s>":
                break

            if order == 1:
                nxt = _sample(uni_items, temperature, top_k, top_p, rng)

            elif order == 2:
                prev1 = tokens[-1]
                nxt = _next_from_bigram(prev1)

            else:  # order == 3 (por defecto)
                prev2 = tokens[-2] if len(tokens) >= 2 else "<s>"
                prev1 = tokens[-1]
                cand = tri_followers(prev2, prev1)
                if cand:
                    nxt = _sample(cand, temperature, top_k, top_p, rng)
                else:
                    # backoff simple a unigramas si no hay contexto
                    nxt = _sample(uni_items, temperature, top_k, top_p, rng)

            tokens.append(nxt)
            if len(tokens) >= max_len - 1 and "</s>" in uni:
                tokens.append("</s>")

        surface = [t for t in tokens if t not in ("<s>", "</s>")]
        return " ".join(surface), tokens

    def close() -> None:
        """Cierra recursos asociados (conexión SQLite)."""
        try:
            tri_conn.close()
        except Exception:
            pass

    return gen, close


# ===============================
# PRUEBAS (documentación breve)
# ===============================
if __name__ == '__main__':
    os.makedirs("quinto-punto", exist_ok=True)

    tests = [
        ("20N", ["the", "this", "i", "if"]),
        ("BAC", ["i", "my", "today", "we"]),
    ]
    configs = [
        # Calidad (trigrama backoff)
        {"order": 3, "temperature": 0.7, "top_k": 60, "top_p": 0.95, "seed": 7},
        {"order": 3, "temperature": 1.0, "top_k": 60, "top_p": 0.95, "seed": 21},
        # Estricto “best-by-perplexity” (unigrama)
        {"order": 1, "temperature": 1.0, "top_k": 0, "top_p": 1.0, "seed": 7},
    ]

    report_lines: List[Dict[str, object]] = []
    for prefix, starters in tests:
        gen, close = build_sentence_generator(prefix)
        print(f"\n--- Samples {prefix} ---")
        for cfg in configs:
            for w in starters:
                s, toks = gen(w, max_len=25, **cfg)  # type: ignore[arg-type]
                print(f"[{prefix}] w='{w}' cfg={cfg} -> {s}")
                report_lines.append({
                    "corpus": prefix, "first_word": w, **cfg,
                    "max_len": 25, "sentence": s, "tokens": toks
                })
        close()

    # Guarda las muestras (documentación de pruebas)
    with open("quinto-punto/sentence_samples.jsonl", "w", encoding="utf-8") as f:
        for r in report_lines:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
    print("\n[OK] Muestras guardadas en quinto-punto/sentence_samples.jsonl")


--- Samples 20N ---
[20N] w='the' cfg={'order': 3, 'temperature': 0.7, 'top_k': 60, 'top_p': 0.95, 'seed': 7} -> the NUM and NUM
[20N] w='this' cfg={'order': 3, 'temperature': 0.7, 'top_k': 60, 'top_p': 0.95, 'seed': 7} -> this is a little more but i dont think that they are not to make a good way of a good number of
[20N] w='i' cfg={'order': 3, 'temperature': 0.7, 'top_k': 60, 'top_p': 0.95, 'seed': 7} -> i dont know about the number of the law
[20N] w='if' cfg={'order': 3, 'temperature': 0.7, 'top_k': 60, 'top_p': 0.95, 'seed': 7} -> if you have any experience with this
[20N] w='the' cfg={'order': 3, 'temperature': 1.0, 'top_k': 60, 'top_p': 0.95, 'seed': 21} -> the first half hour golf game with los angeles and convince me that a lot to learn
[20N] w='this' cfg={'order': 3, 'temperature': 1.0, 'top_k': 60, 'top_p': 0.95, 'seed': 21} -> this is that he would be <UNK> am fairly sure it is the problem is probably not agree with joseph
[20N] w='i' cfg={'order': 3, 'temperature': 1.0, '